In [ ]:
import bare
import hsfm

## Pre-process the imagery

In [ ]:
camera_positions_file_name = 'input_data/targets.csv'
template_directory         = 'input_data/fiducials/nagap/notch'
output_directory           = 'input_data/images'

#### Download thumbnails to disk to read off focal length

In [ ]:
hsfm.batch.download_images_to_disk(camera_positions_file_name, 
                                   output_directory='input_data/thumbnails',
                                   image_type='pid_tn') # pid_tiff, pid_tn, pid_jpeg

#### Detect fiducial markers, crop, and enhance contrast

In [ ]:
hsfm.batch.preprocess_images(template_directory,
                             camera_positions_file_name=camera_positions_file_name,
                             output_directory=output_directory,
                             qc=True)

## Download coarse SRTM reference DEM

In [ ]:
output_directory = 'input_data/reference_dem/'

LLLON = -121.90
LLLAT = 48.65
URLON = -121.60
URLAT = 48.85

In [ ]:
reference_dem = hsfm.utils.download_srtm(LLLON,
                                         LLLAT,
                                         URLON,
                                         URLAT,
                                         output_directory=output_directory,
                                         verbose=False)

#### Optional QC:

In [ ]:
hsfm.plot.plot_dem_from_file(reference_dem)

## Generate input for Metashape

In [ ]:
import pandas as pd

In [ ]:
reference_dem = 'input_data/reference_dem/SRTM3/cache/srtm.vrt'
output_directory = 'input_data'

In [ ]:
df = pd.read_csv(camera_positions_file_name)

In [ ]:
hsfm.batch.calculate_heading_from_metadata(df, 
                                           output_directory=output_directory,
                                           reference_dem=reference_dem,
                                           for_metashape=True)

## Download high resolution reference dem

In [ ]:
reference_dem = 'input_data/reference_dem/lidar/baker.tif'

In [ ]:
import driveanon as da
!mkdir input_data/reference_dem/lidar/
da.save('1ObQyjhYB_fjhvqtBq-vK3CdPoQ1Iauyd', filename=reference_dem)

## Run processing with Metashape

In [ ]:
# hsfm.metashape.authentication('path/to/licence/file.lic')

In [ ]:
image_matching_accuracy = 4
densecloud_quality      = 4

In [ ]:
project_name          = 'easton'
input_path            = 'input_data'
output_path           = 'metashape/'
images_path           = 'input_data/images'
images_metadata_file  = 'input_data/metashape_metadata.csv'
focal_length          = 151.303
pixel_pitch           = 0.02
verbose               = True
rotation_enabled      = True

In [ ]:
project_file, point_cloud_file = hsfm.metashape.images2las(project_name,
                                            images_path,
                                            images_metadata_file,
                                            output_path,
                                            focal_length            = focal_length,
                                            pixel_pitch             = pixel_pitch,
                                            image_matching_accuracy = image_matching_accuracy,
                                            densecloud_quality      = densecloud_quality,
                                            rotation_enabled        = rotation_enabled)

In [ ]:
epsg_code = 'EPSG:'+ hsfm.geospatial.get_epsg_code(reference_dem)
dem = hsfm.asp.point2dem(point_cloud_file, 
                         '--nodata-value','-9999',
                         '--tr','0.5',
                         '--threads', '10',
                         '--t_srs', epsg_code,
                         verbose=verbose)

In [ ]:
clipped_reference_dem = 'metashape/reference_dem_clip.tif'
clipped_reference_dem = hsfm.utils.clip_reference_dem(dem, 
                                                      reference_dem,
                                                      output_file_name = clipped_reference_dem,
                                                      buff_size        = 2000,
                                                      verbose = verbose)

In [ ]:
aligned_dem_file, _ =  hsfm.asp.pc_align_p2p_sp2p(dem,
                                                  clipped_reference_dem,
                                                  output_path,
                                                  verbose = verbose)

In [ ]:
hsfm.utils.dem_align_custom(clipped_reference_dem,
                            aligned_dem_file,
                            output_path,
                            verbose = verbose)